# Test duckdb class

In [6]:
import learn2therm.database

import logging
import time
import duckdb as ddb
logging.basicConfig(level=logging.DEBUG)
logging.info('test')

INFO:root:test


In [2]:
db = learn2therm.database.L2TDatabase.from_files('../data/', './dbs/test_db')

INFO:learn2therm.database:Took 0.005660021305084228m to create taxa table
INFO:learn2therm.database:Took 5.709172435601553m to create protein table
INFO:learn2therm.database:Took 0.05393031438191732m to create taxa pair table
INFO:learn2therm.database:Took 10.110815866788228m to protein pair taxa table
INFO:learn2therm.database:Took 6.946803681055704m to create speed indexes


In [2]:
db=learn2therm.database.L2TDatabase('./dbs/test_db')

In [3]:
db.print_metadata()

Metadata for Learn2Therm database at: ./dbs/test_db
Number of taxa: 16664
Number of taxa with OGT labels: 4424
Number proteins with labeled taxa: 18599024
Number of taxa pairs identified: 13784
Number of protein pair hits meeting minumum requirements: 181500502


In [4]:
db.column_schema

,table_catalog,table_schema,table_name,column_name,ordinal_position,column_default,is_nullable,data_type,character_maximum_length,character_octet_length,...,is_identity,identity_generation,identity_start,identity_increment,identity_maximum,identity_minimum,identity_cycle,is_generated,generation_expression,is_updatable
0,NaN,main,taxa,taxa_index,1,NaN,NO,INTEGER,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,main,taxa,ncbi_taxid,2,NaN,NO,INTEGER,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,main,taxa,record_name,3,NaN,YES,VARCHAR,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,main,taxa,filepath,4,NaN,YES,VARCHAR,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,main,taxa,taxonomy,5,NaN,YES,VARCHAR,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,main,taxa,organism,6,NaN,YES,VARCHAR,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,main,taxa,bacdive_id,7,NaN,YES,INTEGER,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,main,taxa,ogt_scraped_string,8,NaN,YES,VARCHAR,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,main,taxa,seq_16srRNA,9,NaN,YES,VARCHAR,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,main,taxa,len_16s,10,NaN,YES,INTEGER,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
t1 = time.time()
df = db.execute("""
    SELECT 
        protein_pairs.prot_pair_index,
        proteins_m.protein_seq AS meso_seq,
        proteins_t.protein_seq AS thermo_seq,
        taxa_m.ogt AS meso_ogt,
        taxa_t.ogt AS thermo_ogt,
        protein_pairs.scaled_local_symmetric_percent_id
    FROM protein_pairs
    INNER JOIN taxa AS taxa_m ON (protein_pairs.meso_index=taxa_m.taxa_index)
    INNER JOIN taxa AS taxa_t ON (protein_pairs.thermo_index=taxa_t.taxa_index)
    INNER JOIN proteins AS proteins_m ON (protein_pairs.meso_protein_int_index=proteins_m.protein_int_index)
    INNER JOIN proteins AS proteins_t ON (protein_pairs.thermo_protein_int_index=proteins_t.protein_int_index)
    WHERE
        taxa_m.len_16s>1300
        AND taxa_t.len_16s>1300
        AND protein_pairs.local_E_Value< 1e-46
        AND protein_pairs.query_align_cov > 0.95
        AND protein_pairs.subject_align_cov > 0.95
        AND protein_pairs.scaled_local_symmetric_percent_id > 0.7
""")
t2=time.time()
print((t2-t1)/60)

7.879966866970062


In [8]:
print(df.columns)
df.describe()

Index(['prot_pair_index', 'meso_seq', 'thermo_seq', 'meso_ogt', 'thermo_ogt',
       'scaled_local_symmetric_percent_id'],
      dtype='object')


,prot_pair_index,meso_ogt,thermo_ogt,scaled_local_symmetric_percent_id
count,1.738494e+06,1.738494e+06,1.738494e+06,1.738494e+06
mean,9.003368e+07,3.003796e+01,4.634518e+01,7.830483e-01
std,5.249167e+07,4.423389e+00,4.629420e+00,6.432864e-02
min,8.160000e+02,7.000000e+00,4.100000e+01,7.001255e-01
25%,4.459774e+07,2.800000e+01,4.250000e+01,7.310126e-01
50%,8.958291e+07,2.900000e+01,4.500000e+01,7.685590e-01
75%,1.366803e+08,3.250000e+01,5.000000e+01,8.214286e-01
max,1.815004e+08,4.000000e+01,7.800000e+01,1.000000e+00
